<a href="https://colab.research.google.com/github/MuhamadRizky101/Data/blob/main/Search_Engine_TKI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tugas Search Engine
Anggota Kelompok

*   Nama : Muhammad AInur Rofiq, Yusril Andrian M, Muhamad Rizky
*   NIM  : 201810370311146, 201810370311167, 201810370311180

# Crawling Data

In [ ]:
import re
from itertools import chain
import pandas as pd
import nltk
import math
import numpy as np

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords

!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 209 kB 5.3 MB/s 


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = ['https://www.merdeka.com/',
           'https://www.kompasiana.com/',
           'https://talktomeinkorean.com/',
           'https://www.korea.net/',
           'https://www.liputan6.com/',
           'https://infokhs.umm.ac.id',
           'https://www.kompasiana.com/',
           'https://infotech.umm.ac.id/',
           'https://infokhs.umm.ac.id/']

text_list = []
doc_list = []

for l in range(len(url)):
  URL = url[l]
  f = requests.get(URL)
  soup = BeautifulSoup(f.content, 'html.parser') 
  dc = 'D'+f'{l+1:02d}'
  doc_list.append(dc)
  quotes_text=[]
  for link in soup.find_all('p'):
    quote_text = link.text.split()
    quote_text = ' '.join(quote_text)
    quotes_text.append(quote_text)

  quotes_text = ' '.join(quotes_text)
  text_list.append(quotes_text)

In [ ]:
dic = {}
for x in range(len(doc_list)):
  dic[doc_list[x]] = text_list[x]

dic

{'D01': 'Membedah Dua Sisi Rokok: Penerimaan dan Beban Negara Mengenal Sesar Aktif Pemicu Gempa Besar yang Mengelilingi Indonesia Memahami Gempa M 7,4 di NTT yang Picu 97 Kali Guncangan Bentrok di Desa Tamilouw yang Tak Perlu Terjadi Cara Mendeteksi dan Mengonfirmasi Varian Covid-19 Omicron Dana Desa Dibegal KKB Papua Tangis Sedih Haji Agus Salim Dari Balik Layar Muktamar PBNU Bos Jalan Tol Jusuf Hamka Pakai Sepatu Sobek, Hobi Makan di Warteg Kisah Jusuf Hamka Peluk Islam, Buya Hamka Gelar Syukuran Undang 300 Orang Jusuf Hamka Batalkan Rencana Bangun 1.000 Masjid Gara-Gara Omicron, 4.500 Penerbangan Dibatalkan di Seluruh Dunia Saat Natal Kasus Omicron Tambah 11, Total Jadi 19 di Indonesia Seluk Beluk Penanganan Omicron, dari Gejala Hingga Pengobatan 200 Relawan Bencana Semeru Dilindungi BPJS Ketenagakerjaan Ramai Disorot, Ini Video Adegan Syuting di Pengungsian Korban Semeru Panglima TNI Kerahkan Yonzipur Kostrad Bantu Warga Terdampak Erupsi Gunung Semeru PPKM Luar Jawa-Bali Diperpanja

In [ ]:
data = pd.DataFrame({'Doc': dic.keys(), 'Text': dic.values()})
data

,Doc,Text
0,D01,Membedah Dua Sisi Rokok: Penerimaan dan Beban ...
1,D02,Syarat: Akun tervalidasiTema: Asuransi Kesehat...
2,D03,"We have everything you need! Learn with books,..."
3,D04,HOT TOPICS 🎧 Fully vaccinated foreign nat'ls n...
4,D05,Uskup Agung Jakarta Ignatius Kardinal Suharyo...
5,D06,Assalamu'alaikum Warahmatullahi Wabarakatuh Se...
6,D07,Syarat: Akun tervalidasiTema: Asuransi Kesehat...
7,D08,We provide some of the latest information on P...
8,D09,Assalamu'alaikum Warahmatullahi Wabarakatuh Se...


# Preprocessing

In [ ]:
def case_folding(data):
  crawl =[]
  for cr in data.values():
    cr = re.sub(r"[^A-Za-z0-9\-]", " ", cr)
    cr = cr.strip()
    cr = re.sub(r'\s+', ' ', cr)
    cr = cr.lower()
    crawl.append(cr)
  return crawl

def tokenized(data):
  return data.apply(nltk.word_tokenize)

kata = ['yang','dari','pada','dan','pun','akan','adalah']
def removeStopWords(data):
  stop_words = set(kata)
  rm_stopwords =[]
  for st in data:
    st = ' '.join([word for word in st if word not in stop_words])
    rm_stopwords.append(st)
  return rm_stopwords

def stemming(data):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  return [stemmer.stem(tweet) for tweet in data]

In [ ]:
data['case_folding'] = case_folding(dic)
data['tokenized'] = tokenized(data['case_folding'])
data['stop_remove'] = removeStopWords(data['tokenized'])
data['stemmer']= stemming(data['stop_remove'])

In [ ]:
data

,Doc,Text,case_folding,tokenized,stop_remove,stemmer
0,D01,Membedah Dua Sisi Rokok: Penerimaan dan Beban ...,membedah dua sisi rokok penerimaan dan beban n...,"[membedah, dua, sisi, rokok, penerimaan, dan, ...",membedah dua sisi rokok penerimaan beban negar...,bedah dua sisi rokok terima beban negara kenal...
1,D02,Syarat: Akun tervalidasiTema: Asuransi Kesehat...,syarat akun tervalidasitema asuransi kesehatan...,"[syarat, akun, tervalidasitema, asuransi, kese...",syarat akun tervalidasitema asuransi kesehatan...,syarat akun tervalidasitema asuransi sehat onl...
2,D03,"We have everything you need! Learn with books,...",we have everything you need learn with books e...,"[we, have, everything, you, need, learn, with,...",we have everything you need learn with books e...,we have everything you need learn with books e...
3,D04,HOT TOPICS 🎧 Fully vaccinated foreign nat'ls n...,hot topics fully vaccinated foreign nat ls now...,"[hot, topics, fully, vaccinated, foreign, nat,...",hot topics fully vaccinated foreign nat ls now...,hot topics fully vaccinated foreign nat ls now...
4,D05,Uskup Agung Jakarta Ignatius Kardinal Suharyo...,uskup agung jakarta ignatius kardinal suharyo ...,"[uskup, agung, jakarta, ignatius, kardinal, su...",uskup agung jakarta ignatius kardinal suharyo ...,uskup agung jakarta ignatius kardinal suharyo ...
5,D06,Assalamu'alaikum Warahmatullahi Wabarakatuh Se...,assalamu alaikum warahmatullahi wabarakatuh se...,"[assalamu, alaikum, warahmatullahi, wabarakatu...",assalamu alaikum warahmatullahi wabarakatuh se...,assalamu alaikum warahmatullahi wabarakatuh se...
6,D07,Syarat: Akun tervalidasiTema: Asuransi Kesehat...,syarat akun tervalidasitema asuransi kesehatan...,"[syarat, akun, tervalidasitema, asuransi, kese...",syarat akun tervalidasitema asuransi kesehatan...,syarat akun tervalidasitema asuransi sehat onl...
7,D08,We provide some of the latest information on P...,we provide some of the latest information on p...,"[we, provide, some, of, the, latest, informati...",we provide some of the latest information on p...,we provide some of the latest information on p...
8,D09,Assalamu'alaikum Warahmatullahi Wabarakatuh Se...,assalamu alaikum warahmatullahi wabarakatuh se...,"[assalamu, alaikum, warahmatullahi, wabarakatu...",assalamu alaikum warahmatullahi wabarakatuh se...,assalamu alaikum warahmatullahi wabarakatuh se...


In [ ]:
dic_lis = list(dic.keys())
for x in range(len(dic_lis)):
  dic[dic_lis[x]]=data['stemmer'][x]

In [ ]:
dic

{'D01': 'bedah dua sisi rokok terima beban negara kenal sesar aktif picu gempa besar keliling indonesia paham gempa m 7 4 di ntt picu 97 kali guncang bentrok di desa tamilouw tak perlu jadi cara deteksi konfirmasi varian covid-19 omicron dana desa begal kkb papua tangis sedih haji agus salim balik layar muktamar pbnu bos jalan tol jusuf hamka pakai sepatu sobek hobi makan di warteg kisah jusuf hamka peluk islam buya hamka gelar syukur undang 300 orang jusuf hamka batal rencana bangun 1 000 masjid gara-gara omicron 4 500 terbang batal di seluruh dunia saat natal kasus omicron tambah 11 total jadi 19 di indonesia seluk beluk tangan omicron gejala hingga obat 200 rawan bencana meru lindung bpjs ketenagakerjaan ramai sorot ini video adegan syuting di ungsi korban meru panglima tni kerah yonzipur kostrad bantu warga dampak erupsi gunung meru ppkm luar jawa-bali panjang sampai 3 januari 2022 ini daerah ada di level 1 cegah omicron mendagri tito minta kepala daerah optimal ppkm mikro pemkot d

In [ ]:
result = {}

for dicty in dic:
  p = 1
  for val in dic.get(dicty).split():
    rs = str(dicty)+' : '+str(p)
    result.setdefault(val,[])
    if rs in result.get(val):
      p+=1  
      result.get(val).remove(rs)
      rs = str(dicty)+' : '+str(p)

    result.setdefault(val,[]).append((rs))

In [ ]:
result

{'bedah': ['D01 : 1'],
 'dua': ['D01 : 1'],
 'sisi': ['D01 : 1'],
 'rokok': ['D01 : 1'],
 'terima': ['D01 : 1'],
 'beban': ['D01 : 1'],
 'negara': ['D01 : 1', 'D05 : 10'],
 'kenal': ['D01 : 1'],
 'sesar': ['D01 : 1'],
 'aktif': ['D01 : 1'],
 'picu': ['D01 : 1', 'D01 : 2', 'D01 : 14'],
 'gempa': ['D01 : 2', 'D01 : 14'],
 'besar': ['D01 : 1'],
 'keliling': ['D01 : 1'],
 'indonesia': ['D01 : 1', 'D01 : 9', 'D01 : 16', 'D01 : 17', 'D08 : 16'],
 'paham': ['D01 : 1', 'D01 : 14'],
 'm': ['D01 : 2', 'D01 : 14'],
 '7': ['D01 : 2', 'D01 : 14', 'D01 : 27', 'D04 : 31'],
 '4': ['D01 : 2', 'D01 : 7', 'D01 : 14', 'D04 : 8'],
 'di': ['D01 : 3',
  'D01 : 4',
  'D01 : 10',
  'D01 : 11',
  'D01 : 16',
  'D01 : 18',
  'D01 : 24',
  'D01 : 25',
  'D05 : 3',
  'D05 : 4',
  'D05 : 6',
  'D05 : 10',
  'D05 : 11',
  'D06 : 1',
  'D08 : 14',
  'D09 : 1'],
 'ntt': ['D01 : 2', 'D01 : 14'],
 '97': ['D01 : 2', 'D01 : 14'],
 'kali': ['D01 : 2', 'D01 : 14'],
 'guncang': ['D01 : 2', 'D01 : 14'],
 'bentrok': ['D01 : 2'

# Retrieval

In [ ]:
def output(dictionary):
  ky = list(dictionary.keys())
  Quer = []
  dfi = []
  df_dict = pd.DataFrame(columns=dic.keys(), index=ky)

  for x in df_dict.keys():
    df_dict[x] = df_dict[x].fillna(0)
    for y in range(len(ky)):
      for v in dictionary.get(ky[y]):
        tmp = v.split()
        if tmp[0] == x:
          df_dict[tmp[0]].iloc[y]=tmp[-1]

  for x in range(len(df_dict)):
    tmp = df_dict.iloc[x]
    count = 0
    for c in list(tmp):
      if c != 0 :
        count+= 1

    q_count = round(math.log(len(df_dict.keys())/count),1)
    
    Quer.append(q_count)
    dfi.append(count)
  
  df_dict['dfi'] = dfi
  df_dict['log D/df'] = Quer
  df_dict['label'] = ky
  
  return df_dict

output = output(result)
output

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,D01,D02,D03,D04,D05,D06,D07,D08,D09,dfi,log D/df,label
bedah,1,0,0,0,0,0,0,0,0,1,2.2,bedah
dua,1,0,0,0,0,0,0,0,0,1,2.2,dua
sisi,1,0,0,0,0,0,0,0,0,1,2.2,sisi
rokok,1,0,0,0,0,0,0,0,0,1,2.2,rokok
terima,1,0,0,0,0,0,0,0,0,1,2.2,terima
...,...,...,...,...,...,...,...,...,...,...,...,...
developed,0,0,0,0,0,0,0,17,0,1,2.2,developed
paperplay,0,0,0,0,0,0,0,17,0,1,2.2,paperplay
studio,0,0,0,0,0,0,0,17,0,1,2.2,studio
infotech,0,0,0,0,0,0,0,17,0,1,2.2,infotech


In [ ]:
Q = 'Sebagian besar mahasiswa Universitas Muhammadiyah telah fully vaccinated '

In [ ]:
def VSM(query, dictionary):
  q = query.split()
  q = stemming(q)
  df_vsm = pd.DataFrame(columns=dic.keys(), index=q)
  Quer = []
  dfi = []

  for idx in df_vsm.keys():
    df_vsm[idx] = df_vsm[idx].fillna(0)

  for x in dictionary.keys():
    for y in range(len(q)):
      if q[y] == x :
        for z in dictionary.get(q[y]):
          tmp = z.split()
          df_vsm[tmp[0]].iloc[y] = tmp[-1]

  for x in range(len(df_vsm)):
    tmp = df_vsm.iloc[x]
    count = 0
    for c in list(tmp):
      if c != 0 :
        count+= 1

    if count == 0:
      q_count = 0
    else:
      q_count = round(math.log(len(df_vsm.keys())/count),1)
    
    Quer.append(q_count)
    dfi.append(count)
  
  df_vsm['dfi'] = dfi
  df_vsm['log D/df'] = Quer
  #df_vsm['label'] = q


  return df_vsm

    
vsm = VSM(Q,result)
vsm

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,D01,D02,D03,D04,D05,D06,D07,D08,D09,dfi,log D/df
bagi,0,0,0,0,10,0,0,0,0,1,2.2
besar,1,0,0,0,0,0,0,0,0,1,2.2
mahasiswa,0,0,0,0,0,1,0,10,1,3,1.1
universitas,0,0,0,0,0,1,0,10,1,3,1.1
muhammadiyah,0,0,0,0,0,1,0,10,1,3,1.1
telah,21,0,0,0,0,0,0,0,0,1,2.2
fully,0,0,0,1,0,0,0,0,0,1,2.2
vaccinated,0,0,0,134,0,0,0,0,0,1,2.2


In [ ]:
def tfidf (vsm):
  tfidf = []
  df_tfidf = vsm

  for x in df_tfidf.columns[0:-1]:
    for y in range(len(df_tfidf)):
      if df_tfidf[x].iloc[y] != 0:
        tmp = float(df_tfidf[x].iloc[y])*float(df_tfidf['log D/df'].iloc[y])
        df_tfidf[x].iloc[y] = tmp
        
  df_tfidf['Q'] = df_tfidf['log D/df']
  df_tfidf = df_tfidf.drop(df_tfidf.columns[-3:-1], axis=1)
  return df_tfidf

tfidf = tfidf(vsm)
tfidf

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,D01,D02,D03,D04,D05,D06,D07,D08,D09,Q
bagi,0,0,0,0,22,0,0,0,0,2.2
besar,2.2,0,0,0,0,0,0,0,0,2.2
mahasiswa,0,0,0,0,0,1.1,0,11,1.1,1.1
universitas,0,0,0,0,0,1.1,0,11,1.1,1.1
muhammadiyah,0,0,0,0,0,1.1,0,11,1.1,1.1
telah,46.2,0,0,0,0,0,0,0,0,2.2
fully,0,0,0,2.2,0,0,0,0,0,2.2
vaccinated,0,0,0,294.8,0,0,0,0,0,2.2


In [ ]:
def cosine(tfidf, dictionary):
  res = {}
  Q = tfidf['Q'].tolist()  
  for doc_idx in tfidf.columns[0:-1]:
    D = tfidf[doc_idx].tolist()
    cosin = 0
    dij_qj = 0
    square_d = 0
    square_q = 0

    for count_idx in range(len(Q)):
      dij_qj += float(D[count_idx])*float(Q[count_idx])
      square_d += float(D[count_idx])**2
      square_q += float(Q[count_idx])**2
    try :
      cosin = dij_qj / math.sqrt(square_d*square_q)
    except:
      pass    
    if cosin == 0:
      res[doc_idx] = 0
    else:
      res[doc_idx] = round(cosin,2)

  df_cosine = pd.DataFrame({'Documents':list(res.keys()),
                              'Cosine':list(res.values()),
                              'Text':list(dictionary.values())
                              })

  return df_cosine.sort_values('Cosine', ascending=False)

cosine = cosine(tfidf, dic)
cosine
      

,Documents,Cosine,Text
0,D01,0.44,bedah dua sisi rokok terima beban negara kenal...
3,D04,0.42,hot topics fully vaccinated foreign nat ls now...
4,D05,0.42,uskup agung jakarta ignatius kardinal suharyo ...
5,D06,0.36,assalamu alaikum warahmatullahi wabarakatuh se...
7,D08,0.36,we provide some of the latest information on p...
8,D09,0.36,assalamu alaikum warahmatullahi wabarakatuh se...
1,D02,0.00,syarat akun tervalidasitema asuransi sehat onl...
2,D03,0.00,we have everything you need learn with books e...
6,D07,0.00,syarat akun tervalidasitema asuransi sehat onl...
